## Method 1

In [44]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("AzureBlobFileCheck") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

In [17]:
# create a spark session with Azure dependencies
from azure.storage.blob import BlobServiceClient
from io import BytesIO
import pandas as pd

sas_token = "sp=racwdl&st=2025-08-07T15:22:55Z&se=2026-08-06T23:37:55Z&spr=https&sv=2024-11-04&sr=c&sig=ks0K4JyQik%2FYo%2BiIXNTggfiJgSOCVWH3CI7yNNG9N7I%3D"
container_name = "stgippl-azrgt10153"
blob_folder = "uat/inbound/itemmaster/"

# Create blob client and read data
blob_service_client = BlobServiceClient(
    account_url=f"https://{storage_account_name}.blob.core.windows.net",
    credential=sas_token
)

container_client = blob_service_client.get_container_client(container_name)
blobs = list(container_client.list_blobs(name_starts_with=blob_folder))
parquet_files = [blob.name for blob in blobs if blob.name.endswith('.parquet')]

# Read and combine all parquet files
all_dfs = []
for file_name in parquet_files:
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)
    blob_data = blob_client.download_blob().readall()
    temp_df = pd.read_parquet(BytesIO(blob_data))
    all_dfs.append(temp_df)

# Combine and display results
df = pd.concat(all_dfs, ignore_index=True)
print(f"Loaded {len(parquet_files)} files with {df.shape[0]} rows and {df.shape[1]} columns")
display(df.head())

Loaded 2 files with 138951 rows and 74 columns


,Item,ItemDescription,PlanningItem,PlanningItemDescription,StatItem,ReportingItem,SubGBUExtensionDetails,SubGBUExtension,SubGBUExtensionDescription,SUBGBU,...,LocalItem,MetalGroup,SizeRange,AllItem,L4,L5,L6,L3,IsSparePart,DomainFlag
0,791175,Sydney Opera house silver dangle,791175,Sydney Opera house silver dangle,791175,791175,National Icons,12,Travel and Hobbies,1A,...,False,Silver,one size,All,All,All,All,All,False,DP
1,791182,Globe silver clip,791182,Globe silver clip,791182,791182,N/A,03,Clips and Safety Chains,1A,...,False,Silver,one size,All,All,All,All,All,False,DP
2,791242CZ,Hearts silver dangle with cubic zirconia,791242CZ,Hearts silver dangle with cubic zirconia,791242CZ,791242CZ,N/A,06,Love,1A,...,False,Silver,one size,All,All,All,All,All,False,DP
3,791267,Family home silver charm,791267,Family home silver charm,791267,791267,N/A,04,Family and Friends,1A,...,False,Silver,one size,All,All,All,All,All,False,DP
4,791269CZ,Dice silver charm with cubic zirconia,791269CZ,Dice silver charm with cubic zirconia,791269CZ,791269CZ,Hobbies,12,Travel and Hobbies,1A,...,False,Silver,one size,All,All,All,All,All,False,DP


In [43]:
df = spark.createDataFrame(df)
df.show()

+----------+--------------------+------------+-----------------------+----------+-------------+----------------------+---------------+--------------------------+------+-----------------+----------+---------------------+-----+--------------------+--------+--------------------+-------------+----------------------+----------+-------------------+-----------------------+--------------------+--------------------+--------------------+---------------+----------------+------------------+----+-----------------+----------+--------+----------------+----------+---------------+--------------+-------------+---------+------------------+------+--------+-------+-----------+---------+---------+-----------+----------+--------------------+--------------------+---------+------------+-------------------+-------------+-------------+-------------+-------------+-------------+-----------------+-----------+-----------+----------------+---------+----------+--------------+---------+--------------------+---------+--

### Method 2

In [45]:
# create a spark session
from pyspark.sql import SparkSession
from azure.storage.blob import BlobServiceClient
import tempfile
import os

spark = SparkSession.builder \
    .appName("AzureBlobFileCheck") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

# Set log level to reduce verbose output
spark.sparkContext.setLogLevel("ERROR")

# Suppress INFO logs from py4j
import logging
logging.getLogger("py4j").setLevel(logging.ERROR)

# Azure configuration
storage_account_name = "stgipplgt10153"
sas_token = "sp=racwdl&st=2025-08-07T15:22:55Z&se=2026-08-06T23:37:55Z&spr=https&sv=2024-11-04&sr=c&sig=ks0K4JyQik%2FYo%2BiIXNTggfiJgSOCVWH3CI7yNNG9N7I%3D"
container_name = "stgippl-azrgt10153"
blob_folder = "uat/inbound/itemmaster/"

# Create blob client
blob_service_client = BlobServiceClient(
    account_url=f"https://{storage_account_name}.blob.core.windows.net",
    credential=sas_token
)

container_client = blob_service_client.get_container_client(container_name)
blobs = list(container_client.list_blobs(name_starts_with=blob_folder))
parquet_files = [blob.name for blob in blobs if blob.name.endswith('.parquet')]

print(f"Found {len(parquet_files)} parquet files")

# Download files to temp directory and read with Spark
with tempfile.TemporaryDirectory() as temp_dir:
    # Download all parquet files
    for i, file_name in enumerate(parquet_files[:3]):  # Limit to first 3 files for testing
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)
        local_file_path = os.path.join(temp_dir, f"file_{i}.parquet")
        
        with open(local_file_path, "wb") as download_file:
            download_file.write(blob_client.download_blob().readall())
        
        print(f"Downloaded: {file_name.split('/')[-1]}")
    
    # Read all downloaded parquet files with Spark
    df = spark.read.parquet(f"{temp_dir}/*.parquet")
    
    # Display results
    print(f"Loaded data with {df.count()} rows and {len(df.columns)} columns")
    df.show(5)

Found 2 parquet files
Downloaded: part-00000-tid-3673033938457885842-6ea15b70-a13a-486e-a5e6-83e36d31a139-236-1-c000.snappy.parquet
Downloaded: part-00000-tid-3209283240381449950-763d5c7c-4c9e-40e1-9804-66680db9a4d9-215-1-c000.snappy.parquet
Loaded data with 138951 rows and 74 columns
+--------+--------------------+------------+-----------------------+--------+-------------+----------------------+---------------+--------------------------+------+-----------------+----------+---------------------+-----+------------------+--------+-------------------+-------------+----------------------+----------+-------------------+-----------------------+-------------+---------------+------+---------------+----------------+------------------+----+-----------------+----------+--------+--------------+----------+---------------+--------------+-------------+---------+------------------+------+--------+-------+-----------+---------+---------+-----------+----------+-------+--------------------+---------+---

### DuckDB

In [16]:
import duckdb

con = duckdb.connect()
con.execute("SET azure_transport_option_type = 'curl';")
con.execute("INSTALL azure;")
con.execute("LOAD azure;")

folder = "uat/inbound/locationmaster_sp/locationmaster_sp_202508031910/"
storage_account_name = "stgipplgt10153"
container_name = "stgippl-azrgt10153"
sas_token = "sp=racwdl&st=2025-08-07T15:22:55Z&se=2026-08-06T23:37:55Z&spr=https&sv=2024-11-04&sr=c&sig=ks0K4JyQik%2FYo%2BiIXNTggfiJgSOCVWH3CI7yNNG9N7I%3D"

connection_string = f"DefaultEndpointsProtocol=https;AccountName={storage_account_name};SharedAccessSignature={sas_token};EndpointSuffix=core.windows.net"

con.execute("DROP SECRET IF EXISTS azure_secret;")
con.execute(f"""
    CREATE SECRET azure_secret (
        TYPE azure,
        PROVIDER config,
        CONNECTION_STRING '{connection_string}'
    );
""")

df = con.execute(f"SELECT * FROM 'az://{container_name}/{folder}*.parquet'").df()
print(f"Loaded {df.shape[0]} rows and {df.shape[1]} columns")
display(df.head())
con.close()

Loaded 99 rows and 6 columns


,Location,LocationGroup,LocationType,ValidSubLocationFlag,SubLocation,AllLocation
0,BB,Plant,Inhouse-Primary,True,AAA-B04,All
1,MAIN WHs,Plant,MAIN WHs,True,AAA-B04,All
2,MAIN WHs,Plant,MAIN WHs,True,FS-B04,All
3,MAIN WHs,Plant,MAIN WHs,True,RS-MAIN,All
4,BB,Plant,Inhouse-Primary,True,WH-MAIN,All


### DBeaver Compatible SQL

The following SQL commands can be executed directly in DBeaver with a DuckDB connection:

### DBeaver Setup Instructions

**To use this in DBeaver:**

1. **Install DuckDB Driver in DBeaver:**
   - Go to Database → Driver Manager
   - Search for "DuckDB" and install it
   - Or download DuckDB JDBC driver manually

2. **Create New Connection:**
   - Database → New Database Connection
   - Select "DuckDB" 
   - Set database path (or leave empty for in-memory)
   - Test connection

3. **Execute SQL Commands:**
   - Copy the SQL code above
   - Run each command step by step in DBeaver's SQL editor
   - Or run them all at once

**Alternative - Single Query Approach:**

In [ ]:
-- following command needs to run in DBeaver
SET azure_transport_option_type = 'curl';
INSTALL azure; LOAD azure;
DROP SECRET IF EXISTS azure_secret;
CREATE SECRET azure_secret (TYPE azure, PROVIDER config, CONNECTION_STRING 'DefaultEndpointsProtocol=https;AccountName=stgipplgt10153;SharedAccessSignature=sp=racwdl&st=2025-08-07T15:22:55Z&se=2026-08-06T23:37:55Z&spr=https&sv=2024-11-04&sr=c&sig=ks0K4JyQik%2FYo%2BiIXNTggfiJgSOCVWH3CI7yNNG9N7I%3D;EndpointSuffix=core.windows.net');
SELECT * FROM 'az://stgippl-azrgt10153/uat/inbound/locationmaster_sp/locationmaster_sp_202508031910/*.parquet' LIMIT 100;